
# Automatic Pauli string generation

To prepare VQE measurement circuits, the molecular Hamiltonian $\hat{H}$ needs to be represented by products of the Pauli spin operators. A straightforward procedure to do this, is to map the creation and annihilation operators of the second quantized representation of $\hat{H}$ to Pauli operators. Due to the two particle interrelating nature of $\hat{H}$ this expansion scales polynomially ($\sim n_{\text{vir}}^4$) with the system size (e.g., the number of virtual orbitals $n_{\text{vir}}$). Unfortunately, it also requires to map each qubit to individual spin orbitals. So on $N$ qubit devices, the problem size is limited to $N$ spin orbitals. 

A different approach is to represent $\hat{H}$ directly in a many-particle basis $\{|\Phi_i\rangle\}$ via 
$$
\hat{H} = 1\hat{H}1 = \sum_{ij} |\Phi_i\rangle\underbrace{\langle\Phi_i|\hat{H}|\Phi_j\rangle}_{H_{ij}}\langle\Phi_j| = \sum_{ij}H_{ij} |\Phi_i\rangle\langle\Phi_j |
$$
and then to decompose matrix $H$ into products of Pauli operators. This technique has the advantage that many-particle bases composed of $2^N$ many-particle functions can be mapped to $N$ qubits. Furthermore, the basis $\{|\Phi_i\rangle\}$ can be chosen such that all possible qubit states implicitly conserve (i) particle number, (ii) spin multiplicity, and (iii) spatial (molecular) symmetry. A major drawback, however, is that the number of matrix elements $H_{ij}$ scales factorially with the system size (e.g., the number of virtual orbitals). On small quantum devices with a limited number of qubits, the many-particle basis mapping should still be superior to the second quantized mapping.

In this notebook, the automatic (i) generation of matrices $H$ and (ii) decomposition of $H$ into Pauli strings is presented.

## (1) Basic data structures 

The following listings define data structures fundamental to quantum chemistry, which are helpful when calculating Hamiltonian matrix elements $H_{ij}$ by means of the Slater-Condon rules. 

### `SpatialOrbitalIndex`

The most fundamental object in post-Hartree-Fock theory is a single spatial orbital $|\varphi_p(\vec{r})\rangle$, i.e., a single molecular orbital obtained from a restricted Hartree-Fock calculation. In the following listing, this is represented by `SpatialOrbitalIndex` objects.


In [1]:
class SpatialOrbitalIndex:
    '''A single spatial orbital index'''
    def __init__(self, idx:int):
        self.idx = idx
    def __str__(self) -> str:
        return str(self.idx)
    
#examples:
p = SpatialOrbitalIndex(0)
print("Created spatial orbital " + str(p))

Created spatial orbital 0



### `SingleElectronSpin`

The electron's fundamental spin functions $|\alpha\rangle$ or "up" and $|\beta\rangle$ or "down" are represented by `SingleElectronSpin` objects.  

In [2]:
class SingleElectronSpin:
    '''A single electron's spin (either alpha or beta)'''
    def __init__(self, spin:bool):
        self.sz = spin
    
    @classmethod
    def from_str(cls, spin:str):
        if (spin == '+'):
            return cls(False)
        elif (spin == '-'):
            return cls(True)
        else:
            raise ValueError("Wrong string input for SingleElectronSpin.from_str( " + spin + " )")
    
    def __str__(self) -> str:
        if (self.sz == False):
            return "+"
        else:
            return "-"

#examples:
up = SingleElectronSpin(False) #either bool input
down = SingleElectronSpin.from_str("-") #or from string
print("Created spin up: " + str(up) + " and spin down: " + str(down))

Created spin up: + and spin down: -



### `SpinOrbitalIndex`

Each spatial orbital can be combined with either $|\alpha\rangle$ or $|\beta\rangle$ spin functions to yield spin orbitals $|\chi_p\rangle$. These are represented by `SpinOrbitalIndex` objects.


In [3]:
import re

class SpinOrbitalIndex:
    '''A single spin orbital index composed of 
    (i) a spatial orbital index (SpatialOrbitalIndex) 
    (ii) a single spin (SingleElectronSpin)'''
    def __init__(self, idx:SpatialOrbitalIndex, spin:SingleElectronSpin):
        self.mo_idx = idx
        self.spin = spin

    @classmethod
    def from_str(cls, s:str):
        if re.match("^([0-9]+[+|-])", s):
            match = re.findall("^([0-9]+)([+|-])", s)
            return cls(SpatialOrbitalIndex(int(match[0][0])), SingleElectronSpin.from_str(match[0][1]))
        else:
            raise ValueError("Wrong string input for SpinOrbitalIndex.from_str( " + s + " )")

    def integrate_spin(self, other):
        if self.spin.sz == other.spin.sz:
            return 1.0
        return 0.0

    #Transform stored spatial orbital index and spin to unique spin orbital index
    def so_idx(self) -> int:
        return 2*self.mo_idx.idx + self.spin.sz

    def __str__(self) -> str:
        return str(self.mo_idx) + str(self.spin)
    
    #define comparison operators to allow usage in SortedList
    def __hash__(self) -> int:
        return self.so_idx()
    def __eq__(self, other) -> bool:
        return self.so_idx() == other.so_idx()
    def __lt__(self, other) -> bool:
        return self.so_idx() < other.so_idx()
    
#examples 
p_up = SpinOrbitalIndex(p, up)
p_down = SpinOrbitalIndex.from_str("0-")
print("Created spin orbitals " + str(p_up) + " and " + str(p_down))

Created spin orbitals 0+ and 0-



### `SlaterDeterminant`

Fermionic many-particle functions need to be antisymmetric w.r.t. particle transpositions. This can be realized by antisymmetric products of spin orbitals called Slater determinants. In general, a Slater determinant for $n$ particles is given by

$$
|\Psi\rangle = \sqrt{n!}\mathcal{A}\prod_{i=1}^N |\chi_i\rangle \equiv |\chi_1\chi_2\ldots\chi_n\rangle\,,
$$

where $\mathcal{A}$ denotes the antisymmetrizer with 

$$
\mathcal{A} = \frac{1}{n!}\sum_{\hat{P}\in\mathcal{S}_n} (-1)^{p(\hat{P})}\hat{P}\,.
$$

Here, the permutations $\hat{P}$ are elements of the symmetric group, $\mathcal{S}_n$, of order $n$ and $p(\hat{P})$ denotes the parity of permutation $\hat{P}$.
Due to the antisymmetry, it is customary to order the spin orbital labels in ascending order and keep track of an overall sign $s$.
A single Slater determinant therefore caries information about 

1. its occupied spin orbitals $i < j < k < \ldots$, and
2. its sign $s$.

In the following listing, the class `SlaterDeterminant` is defined, which contains a `SortedSet` of `SpinOrbitalIndex` objects called `orbitals` and an integer `sign`. The second quantized creation and annihilation operators are realized by `create(SpinOrbitalIndex)` and `annihilate(SpinOrbitalIndex)` member functions and adjust the determinants sign if needed.


In [4]:
from sortedcontainers import SortedSet
from typing import List

class SlaterDeterminant:
    '''A single Slater determinant composed of 
    (i) a sorted set of spin orbitals 
    (ii) a sign
    To ensure the correct sign, use create/annihilate functions to add or remove orbitals'''

    #initialize from orbital list (possibly unsorted)
    #create orbitals in reversed order. 
    #e.g.( [0+,1-,2+,1+], +1) -> +|1+> -> -|1+, 2+> -> +|1+, 1-, 2+> -> +|0+, 1+, 1-, 2+>
    def __init__(self, orbitals:List[SpinOrbitalIndex] = [], sign:int = +1):
        self.orbitals = SortedSet([])
        self.sign = sign
        for i in orbitals[::-1]:
            self.create(i)

    #construct directly from orbital string e.g. "0+1-2+..."
    #orbitals will be created in reversed order! (..., 2+, 1-, 0+) 
    @classmethod
    def from_str(cls, orbitalstring:str, sign:int = +1):
        if re.match("^([0-9]+[+|-])", orbitalstring):
            matches = re.findall("([0-9]+[+|-])", orbitalstring)
            orbitallist = []
            for i in matches:
                o = SpinOrbitalIndex.from_str(i)
                orbitallist.append(o)
            return cls(orbitallist, sign)
        else:
            raise ValueError("Wrong string input for SlaterDeterminant.from_str( " + orbitalstring + " )")

    def create(self, so:SpinOrbitalIndex):
        if self.sign == 0:
            return #do nothing if determinant vanished due to nilpotency
        if so in self.orbitals: #if orbital is already occupied, determinant vanishes
            self.orbitals.clear() 
            self.sign = 0 #!!! set sign to zero here, to distinguish true zero with Fermi vacuum |>
        else:
            if self.nelectrons() == 0: #if there are no occupied orbitals so far, just insert 
                self.orbitals.add(so)
            else:
                #iterate to position where new orbital would be added 
                it = iter(self.orbitals)
                element = next(it)
                count = 0
                while element < so:
                    count = count + 1
                    try:
                        element = next(it)
                    except StopIteration:
                        break
                #spin orbital would be added after 'count' indices -> multiply sign by (-1)^count
                if (count % 2 == 1):
                    self.sign = -1*self.sign
                self.orbitals.add(so)
    
    def annihilate(self, so:SpinOrbitalIndex):
        if self.sign == 0:
            return #do nothing if determinant vanished due to nilpotency
        if so not in self.orbitals: #if orbital is not occupied, determinant vanishes 
            self.orbitals.clear()
            self.sign = 0 #!!! set sign to zero here, to distinguish true zero with Fermi vacuum |>
        else:
            ann_idx = self.orbitals.index(so) #get index of annihilated orbital
            #orbital is annihilated arfter 'ann_idx' indices -> multiply sign by (-1)^ann_idx
            if (ann_idx % 2 == 1):
                self.sign = -1*self.sign
            self.orbitals.remove(so)


    def nelectrons(self) -> int:
        return len(self.orbitals)

    def __iter__(self):
        self.iter = iter(self.orbitals)
        return self.iter
    def __next__(self):
        return self.iter.__next__()

    def __str__(self):
        output = ""
        if self.sign == -1:
            output = output + "-"
        elif self.sign == 0:
            output = output + "0"
        else:
            output = output + "+"
        if (self.nelectrons() == 0):
            return output + "|>"
        it = iter(self.orbitals)
        element = next(it)
        output = output + "|" + str(element)
        while True:
            try:
                element = next(it)
                output = output + ", " + str(element)
            except StopIteration:
                break
        output = output + ">"
        return output
    
#examples 
det1 = SlaterDeterminant([p_up, p_down], +1)
det2 = SlaterDeterminant.from_str("0+0-", -1)
det3 = SlaterDeterminant.from_str("0-0+", +1) #beware: creation in reversed order! This possibly changes sign
print("Created Slater determinants " + str(det1) + " , " + str(det2) + " , and " + str(det3))

Created Slater determinants +|0+, 0-> , -|0+, 0-> , and -|0+, 0->



### `ConfigurationStateFunction`

In general, single Slater determinants are not eigenfunctions of the total spin operator $\hat{S}^2$ (there are certain exceptions as, e.g., closed-shell singlet determinants). Linear combinations of Slater determinants, which are proper spin eigenfunctions, are called configuration state functions (CSFs). Since $\left[\hat{H}, \hat{S}^2\right] = 0$, the many-particle basis can be further reduced to house only functions with the correct spin symmetry. The following listing provides a comfortable data structure to represent such linear combinations using `dict`s. 


In [5]:
from typing import Dict
import math

class ConfigurationStateFunction(Dict[SlaterDeterminant, float]):
    '''A single configuration state function (linear combination of Slater Determinants) 
    -> inherting from dict mapping SlaterDeterminant -> prefactor (float)'''
    
    @classmethod
    def from_str(cls, stringmap:Dict[str, float]):
        detmap = {}
        for i in stringmap.items():
            detmap[SlaterDeterminant.from_str(i[0])] = i[1]
        return cls(detmap)
    
    def __str__(self):
        output = ""
        it = iter(self.items())
        keyvalue = next(it)
        output += str(keyvalue[1]) + "*" + str(keyvalue[0])
        while True:
            try:
                keyvalue = next(it)
                output += " + " +  str(keyvalue[1]) + "*" + str(keyvalue[0])
            except StopIteration:
                break
        return output
    
#examples 
csf1 = ConfigurationStateFunction({det1 : 1.0})
csf2 = ConfigurationStateFunction.from_str({"0+1-" : +1.0/math.sqrt(2.0), "0-1+" : -1.0/math.sqrt(2.0)})
print ("Created CSFs " + str(csf1) + " and " + str(csf2))

Created CSFs 1.0*+|0+, 0-> and 0.7071067811865475*+|0+, 1-> + -0.7071067811865475*+|0-, 1+>



## (2) Slater-Condon rules

Arbitrary integrals $\langle\Psi|\hat{H}|\Phi\rangle$ of Slater determinants $|\Psi\rangle$ and $|\Phi\rangle$ break down to sums of one- and two-particle MO integrals according to the Slater-Condon rules (see, e.g., https://en.wikipedia.org/wiki/Slater%E2%80%93Condon_rules ). The following listing provides function `slater_condon_rules(bra, ket, moints1, moints2)`, which calculates such integrals for `SlaterDeterminant`s `bra` and `ket` and the 1- and 2-particle MO integrals `moints1` and `moints2`. An additional function `slater_condon_rules_csf` is provided for direct use with `ConfigurationStateFunction` objects.

By default, `pyscf` stores the 2-particle MO integrals in `numpy` arrays using 4-fold index symmetries realized by lower triangular matrices in column major order. To comfortably access these using the full four index notation, the function `access_4s_pyscf_integrals` is provided and used.


In [6]:
import numpy as np
from itertools import tee
import copy

def access_4s_pyscf_integrals(moints2, i:int, j:int, k:int, l:int) -> np.float64:
    '''Comfortable 4 index access to pyscf 2p integrals in mo basis with 4-fold symmetry (chemist notation!!!)'''
    #sort value pairs ij and kl 
    ij = sorted([i,j])
    kl = sorted([k,l])
    #calculate super indices (lower triangular matrix in column major order!)
    idx1 = int((ij[1]+1)*ij[1]/2 + ij[0])
    idx2 = int((kl[1]+1)*kl[1]/2 + kl[0])
    return moints2[idx1, idx2]

def slater_condon_rules(bra:SlaterDeterminant, ket:SlaterDeterminant, moints1, moints2) -> np.float64:
    '''Given two Slater determinants 'bra' and 'ket', and the 1 and 2 particle 
    integrals im MO basis 'moints1' and 'moints2', respectively, calculate <bra|H|ket>''' 
    result = 0.0

    #(1) extract (i) annihilators, (ii) creators, and (iii) common indices for the substitution ket -> bra
    annihilators = ket.orbitals.difference(bra.orbitals) 
    creators = bra.orbitals.difference(ket.orbitals)
    common = bra.orbitals.intersection(ket.orbitals)

    #check Slater Condon rule case (from substitution order)
    if len(annihilators) == 0:
        #Case 1: <Psi|H|Psi> = sum_(i in Psi) <i|h(1)|i> + sum_(i<j in Psi) <ij|g(1,2)|ij-ji>
        it1 = iter(bra)
        while True:
            try:
                so_i = next(it1) #SpinOrbitalIndex used for spin integration later
                i = so_i.mo_idx.idx 
                it1,it2 = tee(it1) #copy iterator using tee
                #<i|i> (spin integration is always 1) 
                result += moints1[i,i] 
                while True:
                    try:
                        so_j = next(it2)
                        j = so_j.mo_idx.idx
                        #<ij|ij> = [ii|jj] (spin integral is always 1)
                        result += access_4s_pyscf_integrals(moints2, i,i,j,j) 
                        #<ij|ji> = [ij|ji]
                        result -= so_i.integrate_spin(so_j) * access_4s_pyscf_integrals(moints2, i,j,j,i)
                    except StopIteration:
                        break
            except StopIteration:
                break
    elif len(annihilators) == 1:
        #Case 2: <Psi|H|Psi_I^A> = <I|h(1)|A> + sum_(i in Psi) <Ii|g(1,2)|Ai-iA>
        so_I = annihilators[0]
        I = so_I.mo_idx.idx
        so_A = creators[0]
        A = so_A.mo_idx.idx
        #<I|A>
        result += so_I.integrate_spin(so_A) * moints1[I,A]
        for so_i in bra.orbitals: 
            i = so_i.mo_idx.idx 
            #<Ii|Ai> = [IA|ii]
            result += so_I.integrate_spin(so_A) * access_4s_pyscf_integrals(moints2, I,A,i,i)
            #<Ii|iA> = [Ii|iA]
            result -= so_I.integrate_spin(so_i) * so_i.integrate_spin(so_A) * access_4s_pyscf_integrals(moints2, I,i,i,A)
    elif len(annihilators) == 2:
        #Case 3: <Psi|H|Psi_IJ^AB> = <IJ|g(1,2)|AB - BA>
        so_I = annihilators[0]
        I = so_I.mo_idx.idx 
        so_J = annihilators[1]
        J = so_J.mo_idx.idx
        so_A = creators[0]
        A = so_A.mo_idx.idx 
        so_B = creators[1] 
        B = so_B.mo_idx.idx 
        #<IJ|AB> = [IA|JB]
        result += so_I.integrate_spin(so_A) * so_J.integrate_spin(so_B) * access_4s_pyscf_integrals(moints2, I,A,J,B)
        #<IJ|BA> = [IB|JA]
        result -= so_I.integrate_spin(so_B) * so_J.integrate_spin(so_A) * access_4s_pyscf_integrals(moints2, I,B,J,A)


    #before returning the result, take care of determinant signs and signs introduced by substitution 
    temp = copy.deepcopy(ket) 
    for i in annihilators:
        temp.annihilate(i)
    for i in creators[::-1]:
        temp.create(i)
    return bra.sign * temp.sign * result


#overhead for CSFs -> calls determinant routine 'slater_condon_rules' for all combinations
def slater_condon_rules_csf(bra:ConfigurationStateFunction, 
                            ket:ConfigurationStateFunction, 
                            moints1, 
                            moints2) -> np.float64:
    result = 0.0
    for i in bra.items():
        for j in ket.items():
            result += i[1]*j[1]*slater_condon_rules(i[0], j[0], moints1, moints2)
    return result
    


## (3) Example: LiH in STO-3G

In the following listings, the methods and data structures implemented in sections (1) and (2) are put to use to generate 2 qubit Pauli strings for the LiH molecule in the STO-3G basis.

### (3.1) Define Molecule and do Hartree-Fock

First, to identify relevant many-particle functions to represent the Hamiltonian, `pyscf` is used to perform a restricted Hartree-Fock calculation using the molecule's spatial symmetry (point group $C_{\infty v}$). The member function `analyze()` is helpful to print MO information such as occupation, orbital energies and irreducible representations.

In [12]:
from pyscf import gto, scf

#(1) define molecule
mol = gto.Mole()
mol.build(
        atom = '''Li 0 0 -1.505; H 0 0 +1.505''',
        unit = 'B',
        basis = 'sto-3g',
        symmetry = 'Coov'
        )

#(2) do Hartree-Fock calculation and obtain MO integrals  
mf = scf.RHF(mol)
mf.conv_tol = 1e-14
mf.conv_tol_grad = 1e-14
mf.max_cycle = 200
mf.kernel()

#print orbital energies, occupancies and irreps
mf.analyze()

converged SCF energy = -7.8620905820129
TODO: total wave-function symmetry for Coov
occupancy for each irrep:     A1  E1x  E1y
                               2    0    0
**** MO energy ****
MO #1 (A1 #1), energy= -2.34859675449772 occ= 2
MO #2 (A1 #2), energy= -0.285881161968396 occ= 2
MO #3 (A1 #3), energy= 0.0782801227497004 occ= 0
MO #4 (E1x #1), energy= 0.163937027517132 occ= 0
MO #5 (E1y #1), energy= 0.163937027517132 occ= 0
MO #6 (A1 #4), energy= 0.549706240344264 occ= 0
 ** Mulliken atomic charges  **
charge of  0Li =      0.32747
charge of  1H =     -0.32747
Dipole moment(X, Y, Z, Debye):  0.00000,  0.00000, -4.85697


((array([1.99940870722514, 0.50862244782932, 0.              ,
         0.              , 0.16450117179157, 1.32746767315397]),
  array([ 0.32746767315397, -0.32746767315397])),
 array([ 0.              ,  0.              , -4.85696556303459]))


### (3.2) Define many-particle basis

If orbital `MO #1` ($|\varphi_1\rangle$) is frozen, i.e., kept occupied by 2 electrons, and orbitals `MO #4`  ($|\varphi_4\rangle$) and `MO #6` ($|\varphi_6\rangle$) are allowed to be occupied by the particles in orbital `MO #2` ($|\varphi_2\rangle$), the five CSFs 
$$
\left\{
|\overline{\varphi_1}\underline{\varphi_1}\overline{\varphi_2}\underline{\varphi_2}\rangle, 
\tfrac{1}{\sqrt{2}}\left(|\overline{\varphi_1}\underline{\varphi_1}\overline{\varphi_2}\underline{\varphi_4}\rangle - |\overline{\varphi_1}\underline{\varphi_1}\underline{\varphi_2}\overline{\varphi_4}\rangle\right), 
|\overline{\varphi_1}\underline{\varphi_1}\overline{\varphi_4}\underline{\varphi_4}\rangle, 
\tfrac{1}{\sqrt{2}}\left(|\overline{\varphi_1}\underline{\varphi_1}\overline{\varphi_2}\underline{\varphi_6}\rangle - |\overline{\varphi_1}\underline{\varphi_1}\underline{\varphi_2}\overline{\varphi_6}\rangle\right), 
|\overline{\varphi_1}\underline{\varphi_1}\overline{\varphi_6}\underline{\varphi_6}\rangle 
\right\}
$$
are possible. Here, the overlined (underlined) MOs shall denote alpha (beta) spin orbital variants. However, since $|\varphi_4\rangle$ transforms as `E1x`, the second CSF from the left can not contribute to the `A1` ground state wave function. The remaining four CSFs can therefore be encoded to 2 qubits using the 4 dimensional matrix representation of $\hat{H}$. 

In the following listing, the many-particle basis is constructed using `ConfigurationStateFunction` objects. Beware that orbital indices now start counting from index $0$.


In [8]:
mpb = [ConfigurationStateFunction.from_str({"0+0-1+1-" : 1.0}),
       ConfigurationStateFunction.from_str({"0+0-1+5-" : 1.0/math.sqrt(2.0), 
                                            "0+0-1-5+" : -1.0/math.sqrt(2.0)}),
       ConfigurationStateFunction.from_str({"0+0-3+3-" : 1.0}),
       ConfigurationStateFunction.from_str({"0+0-5+5-" : 1.0})]

print("Many-particle basis:")
for i in mpb:
    print(i)

Many-particle basis:
1.0*+|0+, 0-, 1+, 1->
0.7071067811865475*+|0+, 0-, 1+, 5-> + -0.7071067811865475*+|0+, 0-, 1-, 5+>
1.0*+|0+, 0-, 3+, 3->
1.0*+|0+, 0-, 5+, 5->



### (3.3) AO -> MO transformation

To calculate all the Hamiltonian matrix elements for the just created many-particle basis, the 1- and 2-particle MO integrals are required. These can be obtained from `pyscf` using the `hf` and `ao2mo` modules. 


In [9]:
from pyscf import ao2mo
from functools import reduce

#do ao -> mo transformation 
h1ao = scf.hf.get_hcore(mol) #get 1p AO integrals 
h1mo = reduce(np.dot, (mf.mo_coeff.T, h1ao, mf.mo_coeff)) #transform to MO basis 
h2mo = ao2mo.full(mol, mf.mo_coeff) #get 2p MO integrals


### (3.4) Hamiltonian matrix representation

It is now straightforward to compute all matrix elements using the `slater_condon_rules_csf` function. Please note that the two zero elements in the Hamiltonian matrix representation arise due to Brillouin's theorem (see, e.g.,  https://en.wikipedia.org/wiki/Brillouin%27s_theorem )

In [10]:
 H = np.ndarray([len(mpb), len(mpb)], dtype=np.float64)
for i in range(0, len(mpb), 1):
    for j in range(i, len(mpb), 1):
        H[i,j] = slater_condon_rules_csf(mpb[i], mpb[j], h1mo, h2mo)
        H[j,i] = H[i,j] #use hermiticity
np.set_printoptions(precision=14, suppress=True)
print("Hamiltonian matrix representation:")
print(H)

Hamiltonian matrix representation:
[[-8.85876832287669  0.                0.02346209013788  0.12384892974705]
 [ 0.               -8.2296103302711  -0.02768300702936  0.01061285809508]
 [ 0.02346209013788 -0.02768300702936 -8.19355577646358  0.01970951819219]
 [ 0.12384892974705  0.01061285809508  0.01970951819219 -7.814564055647  ]]



### (3.5) Decomposition into Pauli string

Finally, the matrix representation of $\hat{H}$ can be decomposed into the desired Pauli string. In the following listing this is done using `pennylane`. Additionally, the string is formatted to be compatible with `qbos`

In [11]:
import pennylane as qml

#do all replacements from dictionary
def replace_all(text:str, dic)->str:
    for before, after in dic.items():
        text = text.replace(before, after)
    return text

#convert hamiltonian to Pauli strings using pennylane
def pauli_decomposition(H) ->str:
    coeffs, obs_list = qml.utils.decompose_hamiltonian(H) #call decomposition routine 
    hamiltonian = qml.Hamiltonian(coeffs, obs_list, simplify = True)
    pauli_string = str(hamiltonian)
    #make string compatible to qbos 
    replacements = {"\n" : "",
                    "[" : "",
                    "]" : "",
                    "(" : "",
                    ")" : "",
                    "I0" : ""}
    pauli_string = replace_all(pauli_string, replacements)
    return pauli_string

pauli_string = pauli_decomposition(H)
print("Pauli string (compatible with qbOS):")
print(pauli_string)

Pauli string (compatible with qbOS):
-8.27412462131459 + 0.009854759096095117 X1+ -0.25203742835554155 Z1+ 0.017037474116481183 X0+ 0.048082961358845655 X0 X1+ 0.006424616021401921 X0 Z1+ -0.07576596838820597 Y0 Y1+ -0.2700647052593024 Z0+ -0.00985475909609449 Z0 X1+ -0.06254156794725341 Z0 Z1
